In [23]:
import json
import logging
import pytz
import pandas as pd
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from kiteconnect import KiteConnect
from selenium import webdriver
import urllib.parse as urlparse
from selenium.webdriver.common.by import By
import time, pyotp
from nsepython import nse_holidays, nsefetch
from datetime import datetime, timedelta
from stocklist import symbols
import sys

# configure logging once at the start of your script
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

zerodha_key = ""
zerodha_secret = ""
zerodha_user = ""
zerodha_password = "#000"
zerodha_totp_secret = ""



In [24]:
kite = KiteConnect(api_key=zerodha_key)

# 1. Open login URL in Selenium
# Auto-download and use correct ChromeDriver version
# Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # comment this if you want to see browser
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

try:
    # Start Selenium with webdriver-manager
    logging.info("Initializing Chrome WebDriver...")
    
    try:
        # Install and setup ChromeDriver
        # Start Selenium with webdriver-manager
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
                          options=chrome_options)
        logging.info("✅ WebDriver initialized successfully!")
        
    except SessionNotCreatedException as e:
        logging.error(f"❌ Error: Chrome version compatibility issue - {e}")
        logging.error("❌ This might be due to browser-driver version mismatch.")
        sys.exit(1)
        
    except WebDriverException as e:
        logging.error(f"❌ Error: WebDriver initialization failed - {e}")
        logging.error("❌ Please check if Chrome is installed and updated.")
        sys.exit(1)
        
    except Exception as e:
        logging.error(f"❌ Unexpected error during WebDriver setup: {e}")
        sys.exit(1)

    # Print login URL with exception handling
    try:
        logging.info(kite.login_url())
        driver.get(kite.login_url())
        wait = WebDriverWait(driver, 15)
        
        # 2. Enter user_id + password
        wait.until(EC.presence_of_element_located((By.ID, "userid"))).send_keys(zerodha_user)
        driver.find_element(By.ID, "password").send_keys(zerodha_password)
        driver.find_element(By.XPATH, "//button[@type='submit']").click()
        logging.info("✅ Login successful.. going to sleep for 3 secs")
        time.sleep(3)
        
    except NameError:
        logging.error("❌ Error: 'kite' object not found or not initialized")
        logging.error("❌ Please ensure the kite object is properly created")
        
    except AttributeError:
        logging.error("❌ Error: 'login_url' method not found in kite object")
        logging.error("❌ Please check if the method name is correct")
        
    except Exception as e:
        logging.error(f"Error retrieving login URL: {e}")
        sys.exit(1)

except NameError:
    logging.error("❌ Error: 'chrome_options' not defined")
    logging.error("❌ Please define chrome_options before initializing WebDriver")
    sys.exit(1)

except Exception as e:
    logging.error(f"❌ Unexpected error in main execution: {e}")
    sys.exit(1)

# 3. Enter TOTP
totp = pyotp.TOTP(zerodha_totp_secret).now()
print("OTP generated :", totp)
# wait.until(EC.presence_of_element_located((By.ID, "totp"))).send_keys(totp)
# driver.find_element(By.XPATH, "/html/body/div[1]/div/div[2]/div[1]/div/div/div[2]/form/div[2]/div/input").send_keys(totp)
driver.find_element(By.ID, "userid").send_keys(totp)
driver.find_element(By.XPATH, "//button[@type='submit']").click()

print("OTP Validated.. going to sleep for 2 secs")
time.sleep(2)

# 4. Extract request_token from redirected URL
current_url = driver.current_url
driver.quit()

# URL looks like: https://your-redirect-url/?status=success&request_token=xxxx
parsed = urlparse.urlparse(current_url)
request_token = urlparse.parse_qs(parsed.query)["request_token"][0]
print("Request Token:", request_token)

# 5. Exchange request_token for access_token
data = kite.generate_session(request_token, api_secret=zerodha_secret)
access_token = data["access_token"]
kite.set_access_token(access_token)
print("Access Token:", access_token)

# Example API call
profile = kite.profile()
print("Logged in as:", profile["user_name"])

2025-09-28 22:04:16,532 - INFO - Initializing Chrome WebDriver...
2025-09-28 22:04:16,543 - INFO - ====== WebDriver manager ======
2025-09-28 22:04:21,087 - INFO - Get LATEST chromedriver version for google-chrome
2025-09-28 22:04:21,319 - INFO - Get LATEST chromedriver version for google-chrome
2025-09-28 22:04:21,370 - INFO - Driver [C:\Users\chand\.wdm\drivers\chromedriver\win64\140.0.7339.207\chromedriver-win32/chromedriver.exe] found in cache
2025-09-28 22:04:26,212 - INFO - ✅ WebDriver initialized successfully!
2025-09-28 22:04:26,214 - INFO - https://kite.zerodha.com/connect/login?api_key=vs36q6a3dlmzw3ot&v=3
2025-09-28 22:04:28,734 - INFO - ✅ Login successful.. going to sleep for 3 secs


OTP generated : 356244
OTP Validated.. going to sleep for 2 secs
Request Token: W0V9SRzwFxWrMkTTztSG1ZmpUYWEmRQC
Access Token: JFAq9gj0X81r708HV5v0bXtce1DE0VwD
Logged in as: Nirav Kirti Shah


In [27]:

""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
""""""""" Check if given date is trading holiday """""""""""
""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
def holiday_check(date):
    # Step 1 : Get list of trading holidays with description
    try:
        df_holidays = pd.json_normalize(nse_holidays()['FO'])
    except Exception as e:
        print("An unexpected error occurred while fetching holidays from NSE:", e)
        print("Skipping holiday check now", e)
        return False
    # print(df_holidays.head(50))

    # Step 2 : Convert into dataframe
    df_holidays['tradingDate'] = pd.to_datetime(df_holidays['tradingDate'])

    # Step 3: Date to check
    check_date = pd.to_datetime(date)  # Example: Gandhi Jayanti

    # Step 4: Check if the date is a holiday
    if check_date in df_holidays['tradingDate'].values:
        logging.info(f"{check_date.date()} is a holiday: {df_holidays[df_holidays['tradingDate'] == check_date]['description'].values[0]}")       
        return True
    else:
        logging.info(f"{check_date.date()} is NOT a holiday.")
        # print(f"{check_date.date()} is NOT a holiday.")
        return False

# prev_week_num - number of week to go back
# offset : 0 - Monday,  1 - Tuesday,  ....., 4 - Friday
def get_nth_working_day(prev_week_num, offset):
    # Today's date
    tz = pytz.timezone('Asia/Kolkata')
    today = datetime.today().now(tz)
    # today = datetime.today()
    # print(today.tzinfo is not None)

    # Weekday: Monday = 0 ... Sunday = 6
    days_since_offset = (today.weekday() - offset) % 7

    # Go back to last working day, then back by (n-1)*7 days
    total_days_back = days_since_offset + (prev_week_num - 1) * 7

    # Last Target day
    target_day = today - timedelta(days=total_days_back)
    return target_day.date()

    
# working days of week calculation
def get_working_days():
    # if today is Monday then give 2 for prev_monday else 3 and so on    
    logging.info(f"Today is : {datetime.today().weekday()}")
    
    if datetime.today().weekday() == 0 or datetime.today().weekday() == 6:
        prev_monday = get_nth_working_day(2, 0)
        last_monday = get_nth_working_day(1, 0)
    else:
        prev_monday = get_nth_working_day(2, 0)
        last_monday = get_nth_working_day(1, 0)

    if datetime.today().weekday() == 4:
        prev_friday = get_nth_working_day(3, 4)
        last_friday = get_nth_working_day(2, 4)
    else:
        prev_friday = get_nth_working_day(2, 4)
        last_friday = get_nth_working_day(1, 4)
    
    logging.info(f"Before holiday check --> {prev_monday}, {prev_friday}, {last_monday}, {last_friday}")

    first_week_open_date = prev_monday
    while (holiday_check(first_week_open_date)):
        first_week_open_date = first_week_open_date + timedelta(days=1)

    last_week_open_date = last_monday
    while (holiday_check(last_week_open_date)):
        last_week_open_date = last_week_open_date + timedelta(days=1)

    first_week_close_date = prev_friday
    while (holiday_check(first_week_close_date)):
        first_week_close_date = first_week_close_date - timedelta(days=1)

    last_week_close_date = last_friday
    while (holiday_check(last_week_close_date)):
        last_week_close_date = last_week_close_date - timedelta(days=1)

    logging.info(f"After holiday check --> {first_week_open_date}, {first_week_close_date}, {last_week_open_date}, {last_week_close_date}")
    
    return first_week_open_date, first_week_close_date, last_week_open_date, last_week_close_date


In [28]:
instruments = kite.instruments("NFO")  # NSE Futures & Options
instruments_df = pd.DataFrame(instruments)
if not instruments_df.empty: 
   # Save to CSV
   formatted = datetime.now().strftime("%d-%b-%Y %H-%M-%S")
   csv_filename = f"zerodha_NFO_original_{formatted}.csv"
   instruments_df.to_csv(csv_filename, index=False)

In [38]:
def get_ltp(symbol: str, exchange: str = "NSE"):
    """
    Get the last traded price for a given symbol.
    Example: get_ltp("TCS"), get_ltp("RELIANCE"), get_ltp("NIFTY 50", "NSE")
    """
    instrument_token = f"{exchange}:{symbol}"
    # print(instrument_token)
    data = kite.ltp([instrument_token])
    return data[instrument_token]["last_price"]
    
# instrument_token = "NSE:ABB"
# data = kite.ltp([instrument_token])
# data[instrument_token]["last_price"]

# instruments_df.head(5)

In [39]:
def get_expiry_date(instruments_df):
      
    # Match records where `name` or `tradingsymbol` contains the symbol
    matched_df = instruments_df[(instruments_df["name"] == "ABB")]

    matched_df.head(5)
    # Ensure expiry is datetime type
    symbol_df = pd.DataFrame()
    symbol_df["expiry"] = pd.to_datetime(matched_df["expiry"])
    
    # Get unique sorted expiries
    expiries = sorted(symbol_df["expiry"].unique())
   
    if not expiries:
        return None  # ✅ No expiries available

    # Pick the nearest expiry first
    selected_expiry = expiries[0]

    # ✅ Convert today's date to Timestamp
    today = pd.Timestamp(datetime.today().date())
    
    # ✅ If nearest expiry is within 5 days, move to next one (if available)
    if (selected_expiry - today).days < 5 and len(expiries) > 1:
        selected_expiry = expiries[1]

    logging.info(f"selected_expiry : {selected_expiry.date()}")
    
    return selected_expiry.date()

In [40]:
def filter_option_strikes(df, symbol: str, min_strike: float, option_type: str = "CE", expiry: str = None):
    """
    Filter option contracts for a given symbol with strike above min_strike.
    Allows filtering CALL (CE) or PUT (PE).
    If expiry is not provided, the nearest expiry will be used.

    Args:
        symbol (str): Underlying symbol (e.g., "TCS")
        min_strike (float): Minimum strike price threshold
        option_type (str): "CE" for Call, "PE" for Put (default = "CE")
        expiry (str, optional): Expiry date in format "YYYY-MM-DD". Defaults to nearest expiry.
    """
    # Validate option type
    if option_type not in ["CE", "PE"]:
        raise ValueError("option_type must be 'CE' or 'PE'")

    # print(df[df["expiry"]])
    if (option_type == "CE"):
        symbol_df = df[(df["name"] == symbol) & 
                   (df["strike"] > min_strike) & 
                   (df["instrument_type"] == option_type) &
                   (df["expiry"] == expiry)]
    else:
        symbol_df = df[(df["name"] == symbol) & 
                   (df["strike"] < min_strike) & 
                   (df["instrument_type"] == option_type) &
                   (df["expiry"] == expiry)]
    
    if symbol_df.empty:
        return pd.DataFrame()  # no results

    # if expiry is None:
    #     # Pick nearest expiry
    #     nearest_expiry = symbol_df["expiry"].min()
    #     filtered = symbol_df[symbol_df["expiry"] == nearest_expiry]
    # else:
    #     filtered = symbol_df[symbol_df["expiry"] == expiry]

    return symbol_df.sort_values(by="strike")

In [41]:
# Build a combined dataframe for all symbols
all_options_df = pd.DataFrame()

option_type="PE"

symbol_counter = 0
expiry_date = get_expiry_date(instruments_df)
if expiry_date is not None:    
    logging.info(f"Total symbols to process for {option_type} is {len(symbols)}")
    for sym in symbols:
        try:       
            last_traded_price = get_ltp(sym, exchange="NSE")          
            filtered_df = filter_option_strikes(instruments_df, sym, min_strike=last_traded_price, option_type=option_type, expiry=expiry_date)  # adjust min_strike & option_type
            if not filtered_df.empty:
                # print(f"Processed {sym} - {last_traded_price}")
                all_options_df = pd.concat([all_options_df, filtered_df])                
                if (symbol_counter % 50 == 0):
                    logging.info(f" ✅ Processing symbol number - {symbol_counter}")
                symbol_counter += 1                    
        except Exception as e:
            # print(f"Skipping {sym}: {e}")
            logging.error(f"Skipping {sym}: {e}")
else:
     logging.error("Expiry is None... So cannot proceed further")

logging.info(f" ✅ Processed all the symbols : {symbol_counter}")
# Reset index
all_options_df.reset_index(drop=True, inplace=True)

formatted = datetime.now().strftime("%d-%b-%Y %H-%M-%S")
csv_filename = f"zerodha_NFO_filtered_{option_type}_options_{formatted}.csv"
all_options_df.to_csv(csv_filename, index=False)


2025-09-28 22:22:10,102 - INFO - selected_expiry : 2025-10-28
2025-09-28 22:22:10,103 - INFO - Total symbols to process for PE is 218
2025-09-28 22:22:10,337 - INFO -  ✅ Processing symbol number - 0
2025-09-28 22:22:25,140 - INFO -  ✅ Processing symbol number - 50
2025-09-28 22:22:37,257 - INFO -  ✅ Processing symbol number - 100
2025-09-28 22:22:47,448 - ERROR - Skipping PEL: 'NSE:PEL'
2025-09-28 22:22:50,056 - INFO -  ✅ Processing symbol number - 150
2025-09-28 22:23:00,555 - INFO -  ✅ Processed all the symbols : 193


In [42]:
# ---------------------------
# Function to fetch OHLC
# ---------------------------

def get_ohlc_last_20_days(instrument_token: int):
    """Fetch OHLC for last 20 days for a given instrument token."""
    to_date = datetime.today()
    from_date = to_date - timedelta(days=20)
    try:
        data = kite.historical_data(
            instrument_token,
            from_date,
            to_date,
            interval="day",
            continuous=False,
            oi=True
        )
        df = pd.DataFrame(data)
        df["instrument_token"] = instrument_token
        
        # Format the date column as YYYY-MM-DD
        df["date"] = pd.to_datetime(df["date"]).dt.strftime("%Y-%m-%d")
        
        return df
    except Exception as e:
        logging.error(f"Error fetching OHLC for {instrument_token} : {e}")        
        return pd.DataFrame()
        


In [43]:
# ---------------------------
# Collect OHLC data for each instrument in filtered_df
# ---------------------------
ohlc_list = []

logging.info(f"✅ Total tokens - {len(all_options_df["instrument_token"].unique())}")
counter = 0
option_type = all_options_df["instrument_type"][0]
for token in all_options_df["instrument_token"].unique():
    ohlc_df = get_ohlc_last_20_days(token)

    if ohlc_df is not None and not ohlc_df.empty:
        ohlc_list.append(ohlc_df)        
        if (counter % 100 == 0):
            logging.info(f"✅ Processing token number - {counter}")
        counter += 1
    else:
        # Append empty placeholder to keep token trace
        placeholder = pd.DataFrame({
            "instrument_token": [token],
            "date": [pd.NaT],
            "open": [None],
            "high": [None],
            "low": [None],
            "close": [None],
            "volume": [None]
        })
        ohlc_list.append(placeholder)
        logging.warning(f"⚠️ No OHLC data for token - {token}, added placeholder")

logging.info(f"Total processed tokens: {counter}")


# Merge into one OHLC DataFrame
ohlc_all_df = pd.concat(ohlc_list, ignore_index=True)

# ---------------------------
# Join with filtered_df to add symbol, expiry, strike, option_type
# ---------------------------
# Ensure filtered_df has these columns:
# tradingsymbol, expiry, strike, instrument_type, instrument_token
daily_ohlc_df = ohlc_all_df.merge(all_options_df[["instrument_token", "expiry", "name", "strike", "instrument_type"]],
    on="instrument_token",
    how="inner"
)

# Rename for clarity
daily_ohlc_df.rename(columns={"instrument_type": "option_type"}, inplace=True)

daily_ohlc_df.head(5)

formatted = datetime.now().strftime("%d-%b-%Y %H-%M-%S")
csv_filename = f"zerodha_NFO_filtered_{option_type}_daily_OHLC_{formatted}.csv"
daily_ohlc_df.to_csv(csv_filename, index=False)

2025-09-28 22:25:05,334 - INFO - ✅ Total tokens - 2345
2025-09-28 22:25:05,588 - INFO - ✅ Processing token number - 0
2025-09-28 22:25:29,219 - INFO - ✅ Processing token number - 100
2025-09-28 22:25:37,347 - ERROR - Error fetching OHLC for 16266498 : 'date'
2025-09-28 22:25:37,358 - WARNING - ⚠️ No OHLC data for token - 16266498, added placeholder
2025-09-28 22:25:37,739 - ERROR - Error fetching OHLC for 14260482 : 'date'
2025-09-28 22:25:37,742 - WARNING - ⚠️ No OHLC data for token - 14260482, added placeholder
2025-09-28 22:25:39,022 - ERROR - Error fetching OHLC for 16421634 : 'date'
2025-09-28 22:25:39,025 - WARNING - ⚠️ No OHLC data for token - 16421634, added placeholder
2025-09-28 22:25:40,245 - ERROR - Error fetching OHLC for 14334722 : 'date'
2025-09-28 22:25:40,246 - WARNING - ⚠️ No OHLC data for token - 14334722, added placeholder
2025-09-28 22:25:45,511 - ERROR - Error fetching OHLC for 16592386 : 'date'
2025-09-28 22:25:45,514 - WARNING - ⚠️ No OHLC data for token - 16592

In [44]:
first_week_open_date, first_week_close_date, last_week_open_date, last_week_close_date = get_working_days()

first_week_open_date = first_week_open_date.strftime("%Y-%m-%d")
first_week_close_date = first_week_close_date.strftime("%Y-%m-%d")
last_week_open_date = last_week_open_date.strftime("%Y-%m-%d")
last_week_close_date = last_week_close_date.strftime("%Y-%m-%d")

# print(daily_ohlc_df['date'][6])
# print(first_week_open_date)

######## Load weekly OHCL for testing ONLY ######
# import pandas as pd
# file_name = "zerodha_NFO_filtered_daily_OHLC_29-Aug-2025 00-00-57.csv"
# daily_ohlc_df = pd.read_csv(file_name)
#############

weekly_ohlc_df = daily_ohlc_df[daily_ohlc_df['date'].isin([first_week_open_date, 
                                                                           first_week_close_date, last_week_open_date, last_week_close_date])]
formatted = datetime.now().strftime("%d-%b-%Y %H-%M-%S")
csv_filename = f"zerodha_NFO_filtered_{option_type}_weekly_OHLC_{formatted}.csv"
weekly_ohlc_df.to_csv(csv_filename, index=False)

2025-09-28 22:34:51,573 - INFO - Today is : 6
2025-09-28 22:34:51,575 - INFO - Before holiday check --> 2025-09-15, 2025-09-19, 2025-09-22, 2025-09-26
2025-09-28 22:34:53,225 - INFO - 2025-09-15 is NOT a holiday.
2025-09-28 22:34:53,873 - INFO - 2025-09-22 is NOT a holiday.
2025-09-28 22:34:54,502 - INFO - 2025-09-19 is NOT a holiday.
2025-09-28 22:34:55,054 - INFO - 2025-09-26 is NOT a holiday.
2025-09-28 22:34:55,055 - INFO - After holiday check --> 2025-09-15, 2025-09-19, 2025-09-22, 2025-09-26


In [45]:
def find_green_bullish_candles(final_df):
    bullish_message = None
    if len(final_df) == 4:
        openFlag = False
        closeFlag = False

        row = final_df.iloc[0]
        if (row['open'] == '0.00') and (row['high'] == '0.00') and (row['low'] == '0.00'):
            final_df.at[final_df.index[0], 'open'] = row['close']

        row = final_df.iloc[1]
        if (row['open'] == '0.00') and (row['high'] == '0.00') and (row['low'] == '0.00'):
            final_df.at[final_df.index[1], 'open'] = row['close']

        row = final_df.iloc[2]
        if (row['open'] == '0.00') and (row['high'] == '0.00') and (row['low'] == '0.00'):
            final_df.at[final_df.index[2], 'open'] = row['close']

        row = final_df.iloc[3]
        if (row['open'] == '0.00') and (row['high'] == '0.00') and (row['low'] == '0.00'):
            final_df.at[final_df.index[3], 'open'] = row['close']

        # print(
        #     f"first_week_open_date - {final_df.iloc[3]['open']}, first_week_close_date - {final_df.iloc[2]['close']}")
        # print(
        #     f"last_week_open_date - {final_df.iloc[1]['open']}, last_week_close_date - {final_df.iloc[0]['close']}")

        # Validate if each week is GREEN candle
        if ((float(final_df.iloc[3]['close']) > float(final_df.iloc[2]['open'])) and
                (float(final_df.iloc[1]['close']) > float(final_df.iloc[0]['open']))):

            # Check if the last week open day price is less than or equal to the first week open day price
            if final_df.iloc[2]['open'] <= final_df.iloc[0]['open']:
                # print("last_week_open_date open is lower than the first_week_open_date.")
                openFlag = True
         
            # Check if the last week close day price is greater than or equal to the first week close day price
            if final_df.iloc[3]['close'] >= final_df.iloc[1]['close']:
                # print("last_week_close_date close is higher than first_week_close_date.")
                closeFlag = True
           
            # print(openFlag, closeFlag)
            if (openFlag & closeFlag):
                # print(final_df)
                bullish_message = f"***** GREEN bullish ****** {final_df.iloc[0]['name']}, {final_df.iloc[0]['strike']}, {final_df.iloc[0]['expiry']} ***** "
                # write_to_log(bullish_message)
        #     else:
        #         bullish_message = f"NOT bullish, {final_df.iloc[0]['strike']}, {final_df.iloc[0]['EXPIRY_DT']}"
        # else:
        #     bullish_message = f"NOT bullish, {final_df.iloc[0]['strike']}, {final_df.iloc[0]['EXPIRY_DT']}"
    # else:
    #     if not final_df.empty:
    #         bullish_message = f"Insufficient data for {final_df.iloc[0]['strike']}, {final_df.iloc[0]['EXPIRY_DT']}"
    #     else:
    #         bullish_message = f"Insufficient data"

    # print(bullish_message)
    return bullish_message


In [46]:
# ####### Load weekly OHCL for testing ONLY ######
# import pandas as pd
# file_name = "zerodha_NFO_filtered_CE_weekly_OHLC_06-Sep-2025 22-03-09.csv"
# weekly_ohlc_df = pd.read_csv(file_name)
# ############

unique_symbols = weekly_ohlc_df['name'].unique()
# unique_strike_prices  = weekly_ohlc_df['strike'].unique()
# unique_strike_prices = ['290.00']

message = "Not Green bullish candle found"
try:
    with open("output.txt", "w") as file_object:
        for symbol in unique_symbols:
            symbol_df = weekly_ohlc_df[weekly_ohlc_df['name'] == symbol]
            unique_strike_prices  = symbol_df['strike'].unique()
            for strike_price in unique_strike_prices:      
                # logging.info(f" Processing {weekly_ohlc_df['name']} for {weekly_ohlc_df['strike']}")
                final_df = symbol_df[symbol_df['strike'] == strike_price]
                # print(final_df)
                # logging.info(f" df size - {len(final_df)}")
                # message.append(f"Line {strike_price}")
                message = find_green_bullish_candles(final_df)
               
                if message is not None:
                    # print(message)
                    file_object.write(f"{message}\n")                  
                
except IOError as e:
 logging.error(f"An I/O error occurred while writing output : {e}")
except Exception as e:
 logging.error(f"Exception occurred while writing output : {e}")

finally:
    file_object.close() 
    logging.info(f"Program completed")


2025-09-28 22:35:09,756 - INFO - Program completed
